In [41]:
import pathlib
from typing import List, Dict

import gym
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import pandas as pd
from IPython.display import display, HTML, Markdown, DisplayObject, Pretty, TextDisplayObject
from lcs.agents.acs2 import Configuration, ACS2
from lcs.metrics import population_metrics
from lcs.strategies.action_selection import EpsilonGreedy, ActionDelay, KnowledgeArray
from tabulate import tabulate
from myst_nb import glue

from src.bayes_estimation import bayes_estimate
from src.decorators import repeat, get_from_cache_or_run
from src.metrics import parse_experiments_results, corridor_transition_knowledge, grid_transition_knowledge
from src.visualization import biased_exploration_colors, PLOT_DPI

NUM_EXPERIMENTS = 50
COLORS = biased_exploration_colors()

plt.ioff()  # turn off interactive plotting

cache_dir = f'{pathlib.Path().absolute()}/cache'
plot_dir = f'{pathlib.Path().absolute()}/plots'


def run_experiment(env_provider, explore_trials, exploit_trials, **conf):
    env = env_provider()
    env.reset()

    cfg = Configuration(**conf)

    explorer = ACS2(cfg)
    metrics_explore = explorer.explore(env, explore_trials)

    exploiter = ACS2(cfg, explorer.population)
    metrics_exploit = explorer.exploit(env, exploit_trials)

    # Parse results into DataFrame
    metrics_df = parse_experiments_results(metrics_explore, metrics_exploit, cfg.metrics_trial_frequency)

    return metrics_df


def extract(combined_list):
    env1_dfs = [result[0] for result in combined_list]
    env2_dfs = [result[1] for result in combined_list]
    env3_dfs = [result[2] for result in combined_list]
    return env1_dfs, env2_dfs, env3_dfs


def average_experiment_runs(runs_dfs: List[pd.DataFrame]) -> pd.DataFrame:
    return pd.concat(runs_dfs).groupby(['trial', 'phase']).mean().reset_index(level='phase')


def plot(epsilon_greedy_df, action_delay_df, knowledge_array_df, op_initial_df,
         env_name,
         num_explore_trials,
         first_knowledge_trials,
         first_population_trials,
         population_ylim,
         text_box_loc,
         plot_filename=None):
    fig = plt.figure(figsize=(14, 8))

    # Layout
    gs = fig.add_gridspec(2, 2, wspace=.25, hspace=.4)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, :])

    # Global title
    fig.suptitle(f'Performance of [{env_name}] environment', fontsize=24)

    # Knowledge
    epsilon_greedy_df['knowledge'][:first_knowledge_trials].plot(label='Epsilon Greedy', c=COLORS['eg'], ax=ax1)
    action_delay_df['knowledge'][:first_knowledge_trials].plot(label='Action Delay', c=COLORS['ad'], ax=ax1)
    knowledge_array_df['knowledge'][:first_knowledge_trials].plot(label='Knowledge Array', c=COLORS['ka'], ax=ax1)
    op_initial_df['knowledge'][:first_knowledge_trials].plot(label='Optimistic Initial Quality', c=COLORS['oiq'],
                                                             ax=ax1)

    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.set_title('Knowledge')
    ax1.set_xlabel('Trial')
    ax1.set_ylabel('Knowledge')
    ax1.axhline(y=100, color='black', linewidth=1, linestyle="--")
    ax1.yaxis.set_major_formatter(mtick.PercentFormatter())

    # Population
    epsilon_greedy_df['population'][:first_population_trials].plot(label='Epsilon Greedy', c=COLORS['eg'], ax=ax2)
    action_delay_df['population'][:first_population_trials].plot(label='Action Delay', c=COLORS['ad'], ax=ax2)
    knowledge_array_df['population'][:first_population_trials].plot(label='Knowledge Array', c=COLORS['ka'], ax=ax2)
    op_initial_df['population'][:first_population_trials].plot(label='Optimistic Initial Quality', c=COLORS['oiq'],
                                                               ax=ax2)

    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.set_xlabel('Trial')
    ax2.set_ylabel('Classifiers')
    ax2.set_title('Classifiers Population')
    ax2.set_ylim(population_ylim)
    ax2.xaxis.set_major_formatter(mtick.FormatStrFormatter('%.0f'))
    ax2.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.0f'))

    # Steps in trial
    window = 3  # window for moving average
    epsilon_greedy_df['steps_in_trial'].rolling(window=window).mean().plot(label='Epsilon Greedy', c=COLORS['eg'],
                                                                           ax=ax3)
    action_delay_df['steps_in_trial'].rolling(window=window).mean().plot(label='Action Delay', c=COLORS['ad'], ax=ax3)
    knowledge_array_df['steps_in_trial'].rolling(window=window).mean().plot(label='Knowledge Array', c=COLORS['ka'],
                                                                            ax=ax3)
    op_initial_df['steps_in_trial'].rolling(window=window).mean().plot(label='Optimistic Initial Quality',
                                                                       c=COLORS['oiq'], ax=ax3)

    ax3.spines['top'].set_visible(False)
    ax3.spines['right'].set_visible(False)
    ax3.set_xlabel('Trial')
    ax3.set_ylabel('Steps')
    ax3.set_title('Steps in trial')
    ax3.axvline(x=num_explore_trials, color='black', linewidth=1, linestyle="--")
    ax3.text(**text_box_loc, s=f'Moving average of {window} samples', style='italic',
             bbox={'facecolor': 'red', 'alpha': 0.2, 'pad': 10})

    # Create legend
    handles, labels = ax3.get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=4)

    if plot_filename:
        fig.savefig(plot_filename, dpi=PLOT_DPI)

# Experiment 1 - Multi-steps problems performance

## Setup

### Corridor

In [42]:
import gym_corridor  # noqa: F401
from src.observation_wrappers import CorridorObservationWrapper


def corridor20_env_provider():
    import gym_corridor  # noqa: F401
    return CorridorObservationWrapper(gym.make(f'corridor-20-v0'))


def corridor40_env_provider():
    import gym_corridor  # noqa: F401
    return CorridorObservationWrapper(gym.make(f'corridor-40-v0'))


def corridor100_env_provider():
    import gym_corridor  # noqa: F401
    return CorridorObservationWrapper(gym.make(f'corridor-100-v0'))


# Function for calculating relevant metrics
def corridor_metrics(agent, env):
    pop = agent.population
    metrics = {
        'knowledge': corridor_transition_knowledge(pop, env)
    }
    metrics.update(population_metrics(pop, env))
    return metrics


corridor_base_params = {
    "classifier_length": 1,
    "number_of_possible_actions": 2,
    "epsilon": 0.8,
    "beta": 0.2,
    "gamma": 0.95,
    "initial_q": 0.5,
    "theta_exp": 50,
    "theta_ga": 50,
    "do_ga": True,
    "mu": 0.03,
    "u_max": 1,
    "metrics_trial_frequency": 1,
    "user_metrics_collector_fcn": corridor_metrics
}

# Start experiments
corridor_explore_trials, corridor_exploit_trials = 60, 20
USE_RAY = True


@get_from_cache_or_run(cache_path=f'{cache_dir}/corridor/epsilon_greedy.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def corridor_epsilon_greedy():
    corridor20 = run_experiment(corridor20_env_provider,
                                corridor_explore_trials,
                                corridor_exploit_trials,
                                **(corridor_base_params | {'action_selector': EpsilonGreedy}))

    corridor40 = run_experiment(corridor40_env_provider,
                                corridor_explore_trials,
                                corridor_exploit_trials,
                                **(corridor_base_params | {'action_selector': EpsilonGreedy}))

    corridor100 = run_experiment(corridor100_env_provider,
                                 corridor_explore_trials,
                                 corridor_exploit_trials,
                                 **(corridor_base_params | {'action_selector': EpsilonGreedy}))

    return corridor20, corridor40, corridor100


@get_from_cache_or_run(cache_path=f'{cache_dir}/corridor/action_delay.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def corridor_action_delay():
    corridor20 = run_experiment(corridor20_env_provider,
                                corridor_explore_trials,
                                corridor_exploit_trials,
                                **(corridor_base_params | {'action_selector': ActionDelay,
                                                           'biased_exploration_prob': 0.5}))

    corridor40 = run_experiment(corridor40_env_provider,
                                corridor_explore_trials,
                                corridor_exploit_trials,
                                **(corridor_base_params | {'action_selector': ActionDelay,
                                                           'biased_exploration_prob': 0.5}))

    corridor100 = run_experiment(corridor100_env_provider,
                                 corridor_explore_trials,
                                 corridor_exploit_trials,
                                 **(corridor_base_params | {'action_selector': ActionDelay,
                                                            'biased_exploration_prob': 0.5}))

    return corridor20, corridor40, corridor100


@get_from_cache_or_run(cache_path=f'{cache_dir}/corridor/knowledge_array.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def corridor_knowledge_array():
    corridor20 = run_experiment(corridor20_env_provider,
                                corridor_explore_trials,
                                corridor_exploit_trials,
                                **(corridor_base_params | {'action_selector': KnowledgeArray,
                                                           'biased_exploration_prob': 0.5}))

    corridor40 = run_experiment(corridor40_env_provider,
                                corridor_explore_trials,
                                corridor_exploit_trials,
                                **(corridor_base_params | {'action_selector': KnowledgeArray,
                                                           'biased_exploration_prob': 0.5}))

    corridor100 = run_experiment(corridor100_env_provider,
                                 corridor_explore_trials,
                                 corridor_exploit_trials,
                                 **(corridor_base_params | {'action_selector': KnowledgeArray,
                                                            'biased_exploration_prob': 0.5}))

    return corridor20, corridor40, corridor100


@get_from_cache_or_run(cache_path=f'{cache_dir}/corridor/oiq.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def corridor_oiq():
    corridor20 = run_experiment(corridor20_env_provider,
                                corridor_explore_trials,
                                corridor_exploit_trials,
                                **(corridor_base_params | {'action_selector': EpsilonGreedy,
                                                           'biased_exploration_prob': 0.8}))

    corridor40 = run_experiment(corridor40_env_provider,
                                corridor_explore_trials,
                                corridor_exploit_trials,
                                **(corridor_base_params | {'action_selector': EpsilonGreedy,
                                                           'biased_exploration_prob': 0.8}))

    corridor100 = run_experiment(corridor100_env_provider,
                                 corridor_explore_trials,
                                 corridor_exploit_trials,
                                 **(corridor_base_params | {'action_selector': EpsilonGreedy,
                                                            'biased_exploration_prob': 0.8}))

    return corridor20, corridor40, corridor100

### Grid

In [43]:
# Function for calculating relevant metrics
def grid_metrics(agent, env):
    pop = agent.population
    metrics = {
        'knowledge': grid_transition_knowledge(pop, env)
    }
    metrics.update(population_metrics(pop, env))
    return metrics


def grid20_env_provider():
    import gym_grid  # noqa: F401
    return gym.make(f'grid-20-v0')


def grid40_env_provider():
    import gym_grid  # noqa: F401
    return gym.make(f'grid-40-v0')


def grid100_env_provider():
    import gym_grid  # noqa: F401
    return gym.make(f'grid-100-v0')


grid_base_params = {
    "classifier_length": 2,
    "number_of_possible_actions": 4,
    "epsilon": 0.8,
    "beta": 0.2,
    "gamma": 0.95,
    "initial_q": 0.5,
    "theta_exp": 50,
    "theta_ga": 50,
    "do_ga": True,
    "mu": 0.03,
    "u_max": 1,
    "metrics_trial_frequency": 1,
    "user_metrics_collector_fcn": grid_metrics
}

# Start experiments
grid_explore_trials, grid_exploit_trials = 60, 20


@get_from_cache_or_run(cache_path=f'{cache_dir}/grid/epsilon_greedy.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def grid_epsilon_greedy():
    grid20 = run_experiment(grid20_env_provider,
                            grid_explore_trials,
                            grid_exploit_trials,
                            **(grid_base_params | {'action_selector': EpsilonGreedy}))

    grid40 = run_experiment(grid40_env_provider,
                            grid_explore_trials,
                            grid_exploit_trials,
                            **(grid_base_params | {'action_selector': EpsilonGreedy}))

    grid100 = run_experiment(grid100_env_provider,
                             grid_explore_trials,
                             grid_exploit_trials,
                             **(grid_base_params | {'action_selector': EpsilonGreedy}))

    return grid20, grid40, grid100


@get_from_cache_or_run(cache_path=f'{cache_dir}/grid/action_delay.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def grid_action_delay():
    grid20 = run_experiment(grid20_env_provider,
                            grid_explore_trials,
                            grid_exploit_trials,
                            **(grid_base_params | {'action_selector': ActionDelay, 'biased_exploration_prob': 0.5}))

    grid40 = run_experiment(grid40_env_provider,
                            grid_explore_trials,
                            grid_exploit_trials,
                            **(grid_base_params | {'action_selector': ActionDelay, 'biased_exploration_prob': 0.5}))

    grid100 = run_experiment(grid100_env_provider,
                             grid_explore_trials,
                             grid_exploit_trials,
                             **(grid_base_params | {'action_selector': ActionDelay, 'biased_exploration_prob': 0.5}))

    return grid20, grid40, grid100


@get_from_cache_or_run(cache_path=f'{cache_dir}/grid/knowledge_array.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def grid_knowledge_array():
    grid20 = run_experiment(grid20_env_provider,
                            grid_explore_trials,
                            grid_exploit_trials,
                            **(grid_base_params | {'action_selector': KnowledgeArray, 'biased_exploration_prob': 0.5}))

    grid40 = run_experiment(grid40_env_provider,
                            grid_explore_trials,
                            grid_exploit_trials,
                            **(grid_base_params | {'action_selector': KnowledgeArray, 'biased_exploration_prob': 0.5}))

    grid100 = run_experiment(grid100_env_provider,
                             grid_explore_trials,
                             grid_exploit_trials,
                             **(grid_base_params | {'action_selector': KnowledgeArray, 'biased_exploration_prob': 0.5}))

    return grid20, grid40, grid100


@get_from_cache_or_run(cache_path=f'{cache_dir}/grid/oiq.dill')
@repeat(num_times=NUM_EXPERIMENTS, use_ray=USE_RAY)
def grid_oiq():
    grid_20 = run_experiment(grid20_env_provider,
                             grid_explore_trials,
                             grid_exploit_trials,
                             **(grid_base_params | {'action_selector': EpsilonGreedy, 'biased_exploration_prob': 0.8}))

    grid_40 = run_experiment(grid40_env_provider,
                             grid_explore_trials,
                             grid_exploit_trials,
                             **(grid_base_params | {'action_selector': EpsilonGreedy, 'biased_exploration_prob': 0.8}))

    grid_100 = run_experiment(grid100_env_provider,
                              grid_explore_trials,
                              grid_exploit_trials,
                              **(grid_base_params | {'action_selector': EpsilonGreedy, 'biased_exploration_prob': 0.8}))

    return grid_20, grid_40, grid_100

## Results

In [44]:
# Execute calculations
corridor20_eg_dfs, corridor40_eg_dfs, corridor100_eg_dfs = extract(corridor_epsilon_greedy())
corridor20_ad_dfs, corridor40_ad_dfs, corridor100_ad_dfs = extract(corridor_action_delay())
corridor20_ka_dfs, corridor40_ka_dfs, corridor100_ka_dfs = extract(corridor_knowledge_array())
corridor20_oiq_dfs, corridor40_oiq_dfs, corridor100_oiq_dfs = extract(corridor_oiq())

# Plot results
plot(
    average_experiment_runs(corridor20_eg_dfs),
    average_experiment_runs(corridor20_ad_dfs),
    average_experiment_runs(corridor20_ka_dfs),
    average_experiment_runs(corridor20_oiq_dfs),
    env_name='Corridor-20',
    num_explore_trials=corridor_explore_trials,
    first_knowledge_trials=30,
    first_population_trials=20,
    population_ylim=(17, 40),
    text_box_loc={"x": 63, "y": 120},
    plot_filename=f'{plot_dir}/corridor-performance.png'
)

:::{figure-md} corridor-fig
:class: full-width
<img src="plots/corridor-performance.png">

Performance in Corridor environment
:::

In [45]:
# Execute calculations
grid20_eg_dfs, grid40_eg_dfs, grid100_eg_dfs = extract(grid_epsilon_greedy())
grid20_ad_dfs, grid40_ad_dfs, grid100_ad_dfs = extract(grid_action_delay())
grid20_ka_dfs, grid40_ka_dfs, grid100_ka_dfs = extract(grid_knowledge_array())
grid20_oiq_dfs, grid40_oiq_dfs, grid100_oiq_dfs = extract(grid_oiq())

# Plot results
plot(
    average_experiment_runs(grid20_eg_dfs),
    average_experiment_runs(grid20_ad_dfs),
    average_experiment_runs(grid20_ka_dfs),
    average_experiment_runs(grid20_oiq_dfs),
    env_name='Grid-20',
    num_explore_trials=grid_explore_trials,
    first_knowledge_trials=10,
    first_population_trials=30,
    population_ylim=(70, 105),
    text_box_loc={"x": 63, "y": 1000},
    plot_filename=f'{plot_dir}/grid-performance.png'
)

:::{figure-md} grid-fig
:class: full-width
<img src="plots/grid-performance.png">

Performance in Grid environment
:::

## Statistical verification

```{admonition} Hypothesis testing
:class: tip
In almost every plots the lines clearly converge (knowledge, population, steps in trial). Maybe what would be beneficial would be to futher investigate larger environment problems (sizes 20/40/100) with fixed number of trials and show the differences (probably in the table like exploration method vs environment).
```

In [46]:
def build_models(dfs: Dict[str, pd.DataFrame], field: str, query_condition: str):
    results = {}

    for name, df in dfs.items():
        data_arr = df.query(query_condition)[field].to_numpy()
        bayes_model = bayes_estimate(data_arr)
        results[name] = (bayes_model['mu'], bayes_model['std'])

    return results


def get_average_reward(dfs: Dict[str, pd.DataFrame]):
    results = {}

    for name, df in dfs.items():
        results[name] = df.query('phase == "exploit"')['reward'].mean()

    return results


experiments_data = {
    'corridor20_eg': pd.concat(corridor20_eg_dfs),
    'corridor40_eg': pd.concat(corridor40_eg_dfs),
    'corridor100_eg': pd.concat(corridor100_eg_dfs),

    'corridor20_ad': pd.concat(corridor20_ad_dfs),
    'corridor40_ad': pd.concat(corridor40_ad_dfs),
    'corridor100_ad': pd.concat(corridor100_ad_dfs),

    'corridor20_ka': pd.concat(corridor20_ka_dfs),
    'corridor40_ka': pd.concat(corridor40_ka_dfs),
    'corridor100_ka': pd.concat(corridor100_ka_dfs),

    'corridor20_oiq': pd.concat(corridor20_oiq_dfs),
    'corridor40_oiq': pd.concat(corridor40_oiq_dfs),
    'corridor100_oiq': pd.concat(corridor100_oiq_dfs),

    'grid20_eg': pd.concat(grid20_eg_dfs),
    'grid40_eg': pd.concat(grid40_eg_dfs),
    'grid100_eg': pd.concat(grid100_eg_dfs),

    'grid20_ad': pd.concat(grid20_ad_dfs),
    'grid40_ad': pd.concat(grid40_ad_dfs),
    'grid100_ad': pd.concat(grid100_ad_dfs),

    'grid20_ka': pd.concat(grid20_ka_dfs),
    'grid40_ka': pd.concat(grid40_ka_dfs),
    'grid100_ka': pd.concat(grid100_ka_dfs),

    'grid20_oiq': pd.concat(grid20_oiq_dfs),
    'grid40_oiq': pd.concat(grid40_oiq_dfs),
    'grid100_oiq': pd.concat(grid100_oiq_dfs)
}


@get_from_cache_or_run(cache_path=f'{cache_dir}/bayes/population.dill')
def build_population_models(dfs: Dict[str, pd.DataFrame]):
    return build_models(dfs, field='population', query_condition=f'trial == {corridor_explore_trials - 1}')


@get_from_cache_or_run(cache_path=f'{cache_dir}/bayes/reliable.dill')
def build_reliable_models(dfs: Dict[str, pd.DataFrame]):
    return build_models(dfs, field='reliable', query_condition=f'trial == {corridor_explore_trials - 1}')


@get_from_cache_or_run(cache_path=f'{cache_dir}/bayes/knowledge.dill')
def build_knowledge_models(dfs: Dict[str, pd.DataFrame]):
    return build_models(dfs, field='knowledge', query_condition=f'trial == {corridor_explore_trials - 1}')


@get_from_cache_or_run(cache_path=f'{cache_dir}/bayes/perf_time.dill')
def build_perf_time_models(dfs: Dict[str, pd.DataFrame]):
    return build_models(dfs, field='perf_time', query_condition=f'phase == "explore"')


population_models = build_population_models(experiments_data)
reliable_models = build_reliable_models(experiments_data)
knowledge_models = build_knowledge_models(experiments_data)
perf_time_models = build_perf_time_models(experiments_data)
avg_rewards = get_average_reward(experiments_data)


def print_bayes_table(name_prefix, population_models, reliable_models, knowledge_models, perf_time_models, avg_rewards):
    print_row = lambda r: f'{round(r[0].mean(), 2)} ± {round(r[0].std(), 2)}'

    key_names = [name for name in experiments_data.keys() if name.startswith(name_prefix)]

    bayes_table_data = [
        ['population of classifiers'] + [print_row(v) for name, v in population_models.items() if name in key_names],
        ['reliable classifiers'] + [print_row(v) for name, v in reliable_models.items() if name in key_names],
        ['knowledge'] + [print_row(v) for name, v in knowledge_models.items() if name in key_names],
        ['trial execution time'] + [print_row(v) for name, v in perf_time_models.items() if name in key_names],
        ['average exploit reward'] + [f'{round(v, 2)}' for name, v in avg_rewards.items() if name in key_names],
    ]

    table = tabulate(bayes_table_data,
                     headers=['', 'Epsilon Greedy', 'Action Delay', 'Knowledge Array', 'Optimistic Initial Quality'],
                     tablefmt="html",
                     stralign='right')
    return HTML(table)

# Add glue for rendering output in tabs
glue("corridor20", print_bayes_table('corridor20', population_models, reliable_models, knowledge_models, perf_time_models, avg_rewards))
glue("corridor40", print_bayes_table('corridor40', population_models, reliable_models, knowledge_models, perf_time_models, avg_rewards))
glue("corridor100", print_bayes_table('corridor100', population_models, reliable_models, knowledge_models, perf_time_models, avg_rewards))
glue("grid20", print_bayes_table('grid20', population_models, reliable_models, knowledge_models, perf_time_models, avg_rewards))
glue("grid40", print_bayes_table('grid40', population_models, reliable_models, knowledge_models, perf_time_models, avg_rewards))
glue("grid100", print_bayes_table('grid100', population_models, reliable_models, knowledge_models, perf_time_models, avg_rewards))

,Epsilon Greedy,Action Delay,Knowledge Array,Optimistic Initial Quality
population of classifiers,38.0 ± 0.0,38.0 ± 0.0,38.0 ± 0.0,38.0 ± 0.0
reliable classifiers,38.0 ± 0.0,38.0 ± 0.0,38.0 ± 0.0,38.0 ± 0.0
knowledge,100.0 ± 0.0,100.0 ± 0.0,100.0 ± 0.0,100.0 ± 0.0
trial execution time,0.02 ± 0.0,0.03 ± 0.0,0.05 ± 0.0,0.02 ± 0.0
average exploit reward,1000.0,1000.0,1000.0,1000.0


,Epsilon Greedy,Action Delay,Knowledge Array,Optimistic Initial Quality
population of classifiers,78.0 ± 0.0,78.0 ± 0.0,78.0 ± 0.0,78.0 ± 0.0
reliable classifiers,78.0 ± 0.0,78.0 ± 0.0,78.0 ± 0.0,78.0 ± 0.0
knowledge,100.0 ± 0.0,100.0 ± 0.0,100.0 ± 0.0,100.0 ± 0.0
trial execution time,0.06 ± 0.0,0.07 ± 0.0,0.09 ± 0.0,0.06 ± 0.0
average exploit reward,949.0,998.0,1000.0,962.0


,Epsilon Greedy,Action Delay,Knowledge Array,Optimistic Initial Quality
population of classifiers,198.0 ± 0.0,198.0 ± 0.0,198.0 ± 0.0,198.0 ± 0.0
reliable classifiers,195.25 ± 0.53,195.78 ± 0.38,196.0 ± 0.0,195.67 ± 0.39
knowledge,98.63 ± 0.27,98.88 ± 0.2,98.98 ± 0.0,98.83 ± 0.21
trial execution time,0.15 ± 0.0,0.16 ± 0.0,0.17 ± 0.0,0.16 ± 0.0
average exploit reward,228.0,222.0,329.0,217.0


### Corridor
```{tabbed} Corridor 20
{glue:}`corridor20`
```

```{tabbed} Corridor 40
{glue:}`corridor40`
```

```{tabbed} Corridor 100
{glue:}`corridor100`
```

### Grid
```{tabbed} Grid 20
{glue:}`grid20`
```

```{tabbed} Grid 40
{glue:}`grid40`
```

```{tabbed} Grid 100
{glue:}`grid100`
```

## Observations
...

---

**Software packages used**

In [50]:
import session_info

session_info.show()